In [6]:
#Import relevant libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [7]:
#Define the hyperparameters
BUFFER_SIZE = 70_000
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [8]:
#Download the dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'],  mnist_dataset['test']


#Scaling the dataset. Fist cast the image to avoid value error
def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 225.
    
    return image, label
    

In [9]:
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

#creating the number of validation dataset from the training dataset
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples

#convert number of the validation sample to an integer to prevent potential issue
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

#get the number data point in our  test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#Shuffle the training and validation dataset
shuffle_train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)


#create the train data
train_data = shuffle_train_and_validation_data.skip(num_validation_samples)

#create the validation data
validation_data = shuffle_train_and_validation_data.take(num_validation_samples)


#batching the training data

train_data = train_data.batch(BATCH_SIZE)

#No need to batch the validation and test data(we are not backpropagating on them ). Hence we will take all at once]
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))




 

In [10]:
#Creating the model
model = Sequential([
        Conv2D(50, 5, activation= 'relu', input_shape=(28,28,1)),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(50, 3, activation= 'relu'),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(10)
])

In [11]:
model.summary(line_length=75)

Model: "sequential_1"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d_2 (Conv2D)               (None, 24, 24, 50)            1300        
                                                                           
 max_pooling2d_2 (MaxPooling2D)  (None, 12, 12, 50)            0           
                                                                           
 conv2d_3 (Conv2D)               (None, 10, 10, 50)            22550       
                                                                           
 max_pooling2d_3 (MaxPooling2D)  (None, 5, 5, 50)              0           
                                                                           
 flatten_1 (Flatten)             (None, 1250)                  0           
                                                                           
 dense_1 (Dense)                 (None, 10)                    125

In [12]:
#describe the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#COmpile the model

model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])


#Stop the model from overfitting ie whenever the validation loss increases
#the code tells the model to stop when the val_loss starts to increase in two subsequent epics

early_stopping = EarlyStopping(
    monitor= 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
)

In [14]:
#Train the model
model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    callbacks = [early_stopping],
    validation_data = validation_data,
    verbose =2
)

Epoch 1/20
422/422 - 33s - loss: 0.2742 - accuracy: 0.9237 - val_loss: 0.0865 - val_accuracy: 0.9717 - 33s/epoch - 79ms/step
Epoch 2/20
422/422 - 31s - loss: 0.0753 - accuracy: 0.9769 - val_loss: 0.0581 - val_accuracy: 0.9847 - 31s/epoch - 74ms/step
Epoch 3/20
422/422 - 31s - loss: 0.0530 - accuracy: 0.9839 - val_loss: 0.0369 - val_accuracy: 0.9882 - 31s/epoch - 74ms/step
Epoch 4/20
422/422 - 34s - loss: 0.0432 - accuracy: 0.9870 - val_loss: 0.0301 - val_accuracy: 0.9898 - 34s/epoch - 79ms/step
Epoch 5/20
422/422 - 35s - loss: 0.0367 - accuracy: 0.9886 - val_loss: 0.0334 - val_accuracy: 0.9897 - 35s/epoch - 82ms/step
Epoch 6/20
422/422 - 36s - loss: 0.0313 - accuracy: 0.9904 - val_loss: 0.0256 - val_accuracy: 0.9912 - 36s/epoch - 86ms/step
Epoch 7/20
422/422 - 36s - loss: 0.0273 - accuracy: 0.9914 - val_loss: 0.0218 - val_accuracy: 0.9925 - 36s/epoch - 85ms/step
Epoch 8/20
422/422 - 36s - loss: 0.0240 - accuracy: 0.9925 - val_loss: 0.0204 - val_accuracy: 0.9917 - 36s/epoch - 86ms/step


In [25]:
#Test the model
test_loss, test_accuracy = model.evaluate(test_data)

print(f'Test Loss: {round(test_loss,2)}. Test Accuracy:{round(test_accuracy*100,2)}%')

1/1 [==============================] - 1s 1s/step - loss: 0.0299 - accuracy: 0.9919
Test Loss: 0.03. Test Accuracy:99.19%


In [ ]:
#Plot the image and result
import matplotlib.pyplot as plt
import numpy as np
